***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Intro Vorlesung 4: Phrasenstrukturgrammatiken


In [66]:
import nltk

## Einführungsbeispiel: Umwandlung CFG in X-Bar-Struktur


In [67]:
#vgl. Aufgabe 1, Thema 4, Wiederholungsaufgaben

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> V NP
 
    NP  -> DET N
    NP  -> DET ADJ N

    DET -> "Der" | "den"
    N   -> "Hund" | "Briefträger"
    ADJ -> "schreienden"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'Der Hund jagt den schreienden Briefträger'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

              S                                  
     ┌────────┴────────┐                          
     │                 VP                        
     │        ┌────────┴───────┐                  
     NP       │                NP                
 ┌───┴───┐    │    ┌───────────┼───────────┐      
DET      N    V   DET         ADJ          N     
 │       │    │    │           │           │      
Der     Hund jagt den     schreienden Briefträger



---

## NP-Adjunkt (ADJ)

- **Umwandlung in X-Bar-Struktur, um rekursive Erweiterung um Adjektive zu ermöglichen:**

*Der Hund jagt den **schreienden schreienden** Briefträger*

- **Gleichzeitig sollen Überproduktionen folgender Art vermieden werden:**

*Der Hund jagt den schreienden schreienden **den** Briefträger*


In [68]:
#X-Bar mit NP-Adjunkt (ADJ):
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
   
    VP -> V NP
    
#NP-SPEZIFIZIERER:    
    NP  -> DET NOM
#NP-ADJUNKT:
    NOM -> ADJ NOM
#NP-KOMPLEMENT (leer):
    NOM -> N
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger"
    ADJ -> "schreienden"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Hund jagt den schreienden schreienden Briefträger'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

                   S                                             
     ┌─────────────┴───────┐                                      
     │                     VP                                    
     │        ┌────────────┴───────┐                              
     │        │                    NP                            
     │        │    ┌───────────────┴───────┐                      
     │        │    │                      NOM                    
     │        │    │       ┌───────────────┴───────┐              
     NP       │    │       │                      NOM            
 ┌───┴───┐    │    │       │               ┌───────┴───────┐      
 │      NOM   │    │       │               │              NOM    
 │       │    │    │       │               │               │      
DET      N    V   DET     ADJ             ADJ              N     
 │       │    │    │       │               │               │      
der     Hund jagt den schreienden     schreienden     Briefträger



In [69]:
#Negativ-Beispiel mit wiederholtem Determinierer (wird nicht erkannt):
sent = 'der Hund jagt den schreienden schreienden den Briefträger'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

## NP-Komplement (Genitiv-NP) und VP-Komplement (Objekt-NP)

In [70]:
#X-Bar mit NP-Komplement (Genititv-NP):
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
#VP-SPEZIFIZIERER (leer):   
    VP  -> VERBAL 
#VP-KOMPLEMENT (transitives Verb):    
    VERBAL -> V NP
        
    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
#neues NP-KOMPLEMENT (Genititv-NP):    
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "schreienden"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Hund jagt den Briefträger der Stadt'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

                   S                               
     ┌─────────────┴───────┐                        
     │                     VP                      
     │                     │                        
     │                   VERBAL                    
     │        ┌────────────┴───────┐                
     │        │                    NP              
     │        │    ┌───────────────┴───┐            
     │        │    │                  NOM          
     │        │    │       ┌───────────┴───┐        
     NP       │    │       │               NP      
 ┌───┴───┐    │    │       │           ┌───┴────┐   
 │      NOM   │    │       │           │       NOM 
 │       │    │    │       │           │        │   
DET      N    V   DET      N          DET       N  
 │       │    │    │       │           │        │   
der     Hund jagt den Briefträger     der     Stadt



In [71]:
#Beispiel mit verschachtelten Genitiv-NPs:
sent = 'der Hund jagt den Briefträger der Stadt der Briefträger'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

         S                                                            
     ┌───┴──────────┐                                                  
     │              VP                                                
     │              │                                                  
     │            VERBAL                                              
     │        ┌─────┴─────────────────┐                                
     │        │                       NP                              
     │        │     ┌─────────────────┴────┐                           
     │        │     │                     NOM                         
     │        │     │         ┌────────────┴────┐                      
     │        │     │         │                 NP                    
     │        │     │         │       ┌─────────┴───┐                  
     │        │     │         │       │            NOM                
     │        │     │         │       │    ┌────────┴───┐              

## mehr VP-Komplemente (Objekt und indirektes Objekt)

- 2 Analysen (Ambiguität):
   - *der*: Genitiv (NP-Komplement) oder Dativ (VP-Komplement)!
   - beide Analysen linguistisch möglich (*übergeben* kann sowohl ditransitiv als auch transitiv verwendet werden)

In [72]:
#X-Bar:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
#neue VP-KOMPLEMENT-Regel (ditransitives Verb: 2 Komplemente):        
    VERBAL -> V NP NP


    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "schreienden"
    V   -> "jagt" | "übergibt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Briefträger übergibt den Hund der Stadt'.split()

print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

2
                              S                          
     ┌────────────────────────┴────────┐                  
     │                                 VP                
     │                                 │                  
     │                               VERBAL              
     │                 ┌──────────┬────┴─────────┐        
     NP                │          NP             NP      
 ┌───┴───────┐         │      ┌───┴────┐     ┌───┴────┐   
 │          NOM        │      │       NOM    │       NOM 
 │           │         │      │        │     │        │   
DET          N         V     DET       N    DET       N  
 │           │         │      │        │     │        │   
der     Briefträger übergibt den      Hund  der     Stadt

                              S                          
     ┌────────────────────────┴────┐                      
     │                             VP                    
     │                             │                      
   

## VP-Adjunkt (NP=Kasusadverbial) und intransitive Verben

- führt zu Überproduktion (Analyse als Komplement von intransitivem Verb: falsch)

In [73]:
#X-Bar:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VERBAL -> V NP NP
#neue VP-KOMPLEMENT-Regel für intransitiv (leer):        
    VERBAL -> V
#neues VP-ADJUNKT (NP als Kasusadverbial):        
    VERBAL -> VERBAL NP

    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N    
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt" | "Tag"
    ADJ -> "schreienden" | "ganzen"
    V   -> "jagt" | "läuft" | "übergibt"
""")

parser = nltk.ChartParser(grammar,trace=0)
sent = 'der Briefträger läuft den ganzen Tag'.split()


print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    


2
                      S                             
     ┌────────────────┴──────────┐                   
     │                           VP                 
     │                           │                   
     │                         VERBAL               
     │                ┌──────────┴──────┐            
     │                │                 NP          
     │                │     ┌───────────┴─────┐      
     NP               │     │                NOM    
 ┌───┴───────┐        │     │           ┌─────┴───┐  
 │          NOM     VERBAL  │           │        NOM
 │           │        │     │           │         │  
DET          N        V    DET         ADJ        N 
 │           │        │     │           │         │  
der     Briefträger läuft  den        ganzen     Tag

                      S                            
     ┌────────────────┴─────────┐                   
     │                          VP                 
     │                          │     

#### mögliche Lösung für Überproduktion: Kategorienerweiterung
- Einführung differenzierender Nonterminale im V-Bereich

In [74]:
#X-Bar:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
#Verwendung neuer PRAETERMINALE:
    VERBAL -> TV NP
    VERBAL -> DTV NP NP
    VERBAL -> IV
    VERBAL -> VERBAL NP

    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt" | "Tag"
    ADJ -> "schreienden" | "ganzen"
#neue POS-PRAETERMINALE (Subdifferenzierung V):
    IV   -> "läuft"
    TV   -> "jagt"
    DTV -> "übergibt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Briefträger läuft den ganzen Tag'.split()

print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

1
                      S                             
     ┌────────────────┴──────────┐                   
     │                           VP                 
     │                           │                   
     │                         VERBAL               
     │                ┌──────────┴──────┐            
     │                │                 NP          
     │                │     ┌───────────┴─────┐      
     NP               │     │                NOM    
 ┌───┴───────┐        │     │           ┌─────┴───┐  
 │          NOM     VERBAL  │           │        NOM
 │           │        │     │           │         │  
DET          N        IV   DET         ADJ        N 
 │           │        │     │           │         │  
der     Briefträger läuft  den        ganzen     Tag



In [75]:
# durch Festlegung 'übergibt' als DTV: auch Auflösung Ambiguität von oben:
sent = 'der Briefträger übergibt den Hund der Stadt'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

                              S                          
     ┌────────────────────────┴────────┐                  
     │                                 VP                
     │                                 │                  
     │                               VERBAL              
     │                 ┌──────────┬────┴─────────┐        
     NP                │          NP             NP      
 ┌───┴───────┐         │      ┌───┴────┐     ┌───┴────┐   
 │          NOM        │      │       NOM    │       NOM 
 │           │         │      │        │     │        │   
DET          N        DTV    DET       N    DET       N  
 │           │         │      │        │     │        │   
der     Briefträger übergibt den      Hund  der     Stadt



## VP-Spezifizierer (AUX) mit Zusatzregel für invertiertes Komplement:


In [77]:
#X-Bar (ausgehend von Grammatik ohne IV/TV/DTV-Differenzierung):
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VERBAL -> V NP NP
#neuer VP-SPEZIFIZIERER (Auxiliar):        
    VP -> AUX VERBAL
#neue VP-KOMPLEMENT-REGEL mit invertierter Wortstellung: 
    VERBAL -> NP V

    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "schreienden"
    V   -> "jagt" | "übergibt"
#neue lexikalische Regeln für AUX:
    AUX -> "hat"
    V -> "übergeben"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Briefträger hat den Hund übergeben'.split()

print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

1
                     S                          
     ┌───────────────┴───────┐                   
     │                       VP                 
     │               ┌───────┴────┐              
     │               │          VERBAL          
     │               │       ┌────┴────────┐     
     NP              │       NP            │    
 ┌───┴───────┐       │   ┌───┴────┐        │     
 │          NOM      │   │       NOM       │    
 │           │       │   │        │        │     
DET          N      AUX DET       N        V    
 │           │       │   │        │        │     
der     Briefträger hat den      Hund  übergeben

